In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from data_augmentation import random_transform


from keras.models import Sequential, Model
from keras.optimizers import Adam
from keras.layers import Dense, Conv2D, MaxPooling2D, Reshape, Flatten, Input, merge
from keras import backend as K

Using TensorFlow backend.


In [2]:
resize_shape = (128, 128)

In [3]:
data = pd.read_csv("data/train.csv")

In [4]:
#For now, we remove new_whale
data = data[data['Id'] != 'new_whale'].reset_index(drop=True)

In [5]:
len(data)

9040

# Look at example 

In [6]:
from PIL import Image
image = Image.open('data/train/0a5c0f48.jpg')

In [7]:
image = image.resize(resize_shape)

In [8]:
# plt.imshow(image)
# plt.show()

In [9]:
np.array(image).shape

(128, 128, 3)

In [10]:
gray = np.mean(image, -1)

In [11]:
gray.shape

(128, 128)

In [12]:
# plt.imshow(random_transform(gray),cmap='gray')
# plt.show()

# All images (if small) can be held in memory.

In [13]:
file_list = data['Image']

In [14]:
def get_image(file, shape=resize_shape):
    image = Image.open('data/train/' + file)
    image = image.resize(shape)
    image = np.array(image)
    if len(image.shape) == 3:
        image = np.mean(image, -1) 
    return image

In [15]:
image_list = [get_image(f) for f in file_list]

In [16]:
data['image_array'] = image_list

In [17]:
data.head(4)

,Image,Id,image_array
0,00022e1a.jpg,w_e15442c,"[[196, 191, 195, 197, 206, 204, 219, 221, 214,..."
1,000466c4.jpg,w_1287fbc,"[[189.333333333, 187.333333333, 191.333333333,..."
2,00087b01.jpg,w_da2efe0,"[[191.0, 192.0, 186.0, 195.0, 217.0, 216.0, 21..."
3,001296d5.jpg,w_19e5482,"[[159.333333333, 161.333333333, 162.333333333,..."


# Create Test and Train

In [18]:
from sklearn.utils import shuffle
data = shuffle(data)

test_proportion = 0.8
cutoff_index = int(len(data) * test_proportion)

training_data = data.iloc[:cutoff_index].reset_index(drop=True)
test_data = data.iloc[cutoff_index:].reset_index(drop=True)

In [19]:
from collections import Counter

training_counts = Counter(training_data['Id'])
training_data['Id_count'] = training_data.apply(lambda x: training_counts.get(x["Id"]), axis=1)

test_counts = Counter(test_data['Id'])
test_data['Id_count'] = test_data.apply(lambda x: test_counts.get(x["Id"]), axis=1)

In [20]:
from random import randint

# Create generator

In [21]:
def get_different_category_example(data):
    index_1 = randint(0,len(data)-1)
    image_1 = data['image_array'][index_1]
    id_1 = data['Id'][index_1]
    id_2 = id_1
    while (id_1 == id_2):
        index_2 = randint(0,len(data)-1)
        id_2 = data['Id'][index_2]
    
    image_2 = data['image_array'][index_2]
    
    image_1 = random_transform(image_1)
    image_2 = random_transform(image_2)
    return image_1, image_2

def get_same_category_example(data):
    filtered = data[data['Id_count'] > 1].reset_index(drop=True)
    index_1 = randint(0,len(filtered)-1)
    image_1 = filtered['image_array'][index_1]
    id_1 = filtered['Id'][index_1]   
    id_2 = id_1
    relevant_indices = list(filtered.index[filtered['Id'] == id_1])
    relevant_indices.remove(index_1)
    index_2 = np.random.choice(relevant_indices)
    image_2 = filtered['image_array'][index_2]
    
    image_1 = random_transform(image_1)
    image_2 = random_transform(image_2)
    return image_1, image_2

In [22]:
def binary_data_generator(batch_size, data, resize_shape, p=0.5):
    while True:
        targets = np.random.randint(2, size=batch_size)
        image_1_batch = np.zeros((batch_size, resize_shape[0],resize_shape[1]))
        image_2_batch = np.zeros((batch_size, resize_shape[0],resize_shape[1]))
        for i in range(batch_size):
            if(targets[i]):
                image_1_batch[i,:,:], image_2_batch[i,:,:] = get_same_category_example(data)
            else:
                image_1_batch[i,:,:], image_2_batch[i,:,:] = get_different_category_example(data)

        pairs = [image_1_batch, image_2_batch]

        yield pairs, targets 

# Create the network

In [23]:
input_shape = resize_shape
left_input = Input(input_shape)
right_input = Input(input_shape)

convnet = Sequential()
convnet.add(Reshape((resize_shape[0],resize_shape[1], 1,), input_shape=(resize_shape[0],resize_shape[1],),name='Reshape'))
convnet.add(Conv2D(filters=8, input_shape=(resize_shape[0],resize_shape[1],1,), kernel_size=3, activation='relu',name='conv_1'))
convnet.add(Conv2D(filters=16, kernel_size=3, activation='relu',name='conv_2'))
convnet.add(MaxPooling2D(pool_size=2,name='pool_1'))
convnet.add(Conv2D(filters=32, kernel_size=3, activation='relu',name='conv_3'))
convnet.add(Conv2D(filters=64, kernel_size=3, activation='relu',name='conv_4'))
convnet.add(MaxPooling2D(pool_size=2,name='pool_2'))
convnet.add(Conv2D(filters=128, kernel_size=3, activation='relu',name='conv_5'))
convnet.add(Conv2D(filters=256, kernel_size=3, activation='relu',name='conv_6'))
convnet.add(MaxPooling2D(pool_size=2,name='pool_3'))
convnet.add(Flatten())
convnet.add(Dense(units=1024, activation='relu',name='dense_1'))

encoded_l = convnet(left_input)
encoded_r = convnet(right_input)
L1_distance = lambda x: K.abs(x[0]-x[1])
both = merge([encoded_l,encoded_r], mode = L1_distance, output_shape=lambda x: x[0])
prediction = Dense(1,activation='sigmoid')(both)
siamese_net = Model(input=[left_input,right_input],output=prediction)

/opt/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:22: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
/opt/anaconda3/lib/python3.5/site-packages/keras/legacy/layers.py:458: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  name=name)
/opt/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:24: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("de..., inputs=[<tf.Tenso...)`


In [24]:
LEARNING_RATE = 0.0001
siamese_net.compile(loss="binary_crossentropy",optimizer=Adam(LEARNING_RATE), metrics=['accuracy'])

# Run the network

In [25]:
BATCH_SIZE = 8
training_data_generator = binary_data_generator(BATCH_SIZE, training_data, resize_shape)

In [32]:
history = siamese_net.fit_generator(training_data_generator, verbose=1, epochs=10, steps_per_epoch=100)

Epoch 1/10
 11/100 [==>...........................] - ETA: 2:00 - loss: 0.6498 - acc: 0.6477

KeyboardInterrupt: 

# Evaluation 

In [27]:
evaluation_data_generator = binary_data_generator(BATCH_SIZE, test_data, resize_shape)

evaluation_steps = 100
metric_names = siamese_net.metrics_names
metric_values = siamese_net.evaluate_generator(evaluation_data_generator, steps=evaluation_steps)
for i in range(len(siamese_net.metrics_names)):
    print(metric_names[i], ": ", metric_values[i], sep="")

loss: 0.664925300777
acc: 0.6275


# Save weights

In [33]:
import time
import os

now = time.strftime('%Y.%m.%d %H:%M:%S')
directory = "weights/" + now + "/"
if not os.path.exists(directory):
    os.makedirs(directory)


siamese_net.save_weights(directory + "siamese_weights")
convnet.save_weights(directory + "convnet_weights")

# Sub

In [99]:
def predict_category(image, model, training_images, training_categories):
    best_score = 0
    predicted_category = None
    for i in range(len(training_images)):
        current_score = model.predict([np.expand_dims(image,0), np.expand_dims(training_images[i],0)])[0][0]
        if current_score > best_score:
            best_score = current_score
            predicted_category = training_categories[i]
    return predicted_category
            

In [104]:
#Same image should return 1. Investigate this?

image_1 = training_data['image_array'][0]
image_1 = np.expand_dims(image_1,0)
image_2 = training_data['image_array'][0]
image_2 = np.expand_dims(image_2,0)

siamese_net.predict([image_1,image_2])[0][0]

0.50368077

In [101]:
training_data.head(4)

,Image,Id,image_array,Id_count
0,8769646d.jpg,w_ffda8b2,"[[66, 73, 59, 57, 84, 103, 95, 92, 84, 82, 71,...",4
1,7a99c7df.jpg,w_4fd48e7,"[[232.0, 232.0, 232.0, 232.0, 232.0, 232.0, 23...",6
2,9794021e.jpg,w_6b2995b,"[[167.666666667, 157.666666667, 164.666666667,...",1
3,b137da48.jpg,w_871e32e,"[[132.666666667, 125.666666667, 141.666666667,...",1


In [102]:
image_1 = training_data['image_array'][0]

In [103]:
print(predict_category(image_1, siamese_net, training_data['image_array'], training_data['Id']))

w_60cc444


In [105]:
x = np.stack(training_data['image_array'])

In [106]:
x.shape

(7232, 128, 128)

In [117]:
image_1 = training_data['image_array'][0]
image_1 = np.expand_dims(image_1,0)

image_1.shape

(1, 128, 128)

In [120]:
y = np.repeat(image_1, 7232, axis = 0)

In [121]:
y.shape

(7232, 128, 128)

In [122]:
siamese_net.predict([x,y])

array([[ 0.50368077],
       [ 0.63678646],
       [ 0.73636901],
       ..., 
       [ 0.61671418],
       [ 0.49561834],
       [ 0.63139528]], dtype=float32)